In [21]:
import pandas as pd
import wmfdata as wmf
from wmfdata.utils import pd_display_all

```
"tests": [ 
  {
    "name": "growth-homepage",
    "group": "variant-1"
  },
  {
    "name": "growth-help-panel",
    "group": "variant-2"
  }
]
```

In [11]:
raw_spark = wmf.spark.run("""
SELECT
  event.tests AS tests
FROM event.inukapageview
WHERE
  revision = 21084675
""")

In [12]:
raw_spark.head()

,tests
0,"[(2021-KaiOS-app-homepage-content-suggestions,..."
1,"[(2021-KaiOS-app-homepage-content-suggestions,..."
2,"[(2021-KaiOS-app-homepage-content-suggestions,..."
3,"[(2021-KaiOS-app-homepage-content-suggestions,..."
4,"[(2021-KaiOS-app-homepage-content-suggestions,..."


In [13]:
spark_value = raw_spark.iloc[0, 0]
spark_value

[Row(name='2021-KaiOS-app-homepage-content-suggestions', group='trending-articles')]

In [15]:
spark_value[0]["name"]

'2021-KaiOS-app-homepage-content-suggestions'

In [4]:
raw_presto = wmf.presto.run("""
SELECT
  event.tests AS tests
FROM event.inukapageview
WHERE
  revision = 21084675
""")

raw_presto.head()

,tests
0,"[[2021-KaiOS-app-homepage-content-suggestions,..."
1,"[[2021-KaiOS-app-homepage-content-suggestions,..."
2,"[[2021-KaiOS-app-homepage-content-suggestions,..."
3,"[[2021-KaiOS-app-homepage-content-suggestions,..."
4,"[[2021-KaiOS-app-homepage-content-suggestions,..."


In [17]:
presto_value = presto_raw.iloc[0, 0]
presto_value

[['2021-KaiOS-app-homepage-content-suggestions', 'control']]

In [18]:
presto_value[0][0]

'2021-KaiOS-app-homepage-content-suggestions'

In [19]:
presto_value[0]["name"]

TypeError: list indices must be integers or slices, not str

In [ ]:
presto_extracted = wmf.presto.run("""
SELECT
  event.tests AS tests,
  REDUCE(
    event.tests, 
    NULL, 
    (i, x) -> IF(x.name = '2021-KaiOS-app-homepage-content-suggestions', x."group", i),
    x -> x
  ) AS experiment_group
FROM event.inukapageview
WHERE
  CARDINALITY(
    FILTER(
      event.tests,
      x -> x.name = '2021-KaiOS-app-homepage-content-suggestions'
  )) > 0
""")

In [23]:
presto_extracted.sample(10).pipe(pd_display_all)

,tests,experiment_group
493,"[[2021-KaiOS-app-homepage-content-suggestions, control]]",control
1111,"[[2021-KaiOS-app-homepage-content-suggestions, trending-articles]]",trending-articles
1619,"[[2021-KaiOS-app-homepage-content-suggestions, control]]",control
3,"[[2021-KaiOS-app-homepage-content-suggestions, curated-topics]]",curated-topics
800,"[[2021-KaiOS-app-homepage-content-suggestions, trending-articles]]",trending-articles
1533,"[[2021-KaiOS-app-homepage-content-suggestions, trending-articles]]",trending-articles
1214,"[[2021-KaiOS-app-homepage-content-suggestions, curated-topics]]",curated-topics
577,"[[2021-KaiOS-app-homepage-content-suggestions, trending-articles]]",trending-articles
148,"[[2021-KaiOS-app-homepage-content-suggestions, curated-topics]]",curated-topics
660,"[[2021-KaiOS-app-homepage-content-suggestions, control]]",control


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 1166, in run
    self.function(*self.args, **self.kwargs)
TypeError: stop_session() missing 1 required positional argument: 'session'



```
"tests": {
  "2021-growth-homepage": "variant-1",
  "2021-growth-help-panel: "variant-2"
}
```